In [1]:
#this is the block of code that generates unrealized profits for companies for which portfolia holder has bought 
#and sold the stocks both.

import pandas as pd
from fpdf import FPDF
import yfinance as yf

# Load the Excel file
df_xlsx = pd.read_excel("Book2.xlsx")

# Filter buy and sell data
data_for_buy = df_xlsx[df_xlsx['TYPE'] == 'Buy']
data_for_sell = df_xlsx[df_xlsx['TYPE'] == 'Sell']

# Summarize the quantities by ticker
buy_summary = data_for_buy.groupby('EXCHANGE CODE: TICKER')['UNITS'].sum().to_dict()
sell_summary = data_for_sell.groupby('EXCHANGE CODE: TICKER')['UNITS'].sum().to_dict()
buy_price_dict = data_for_buy.groupby('EXCHANGE CODE: TICKER')['AMOUNT'].sum().to_dict()
sell_price_dict = data_for_sell.groupby('EXCHANGE CODE: TICKER')['AMOUNT'].sum().to_dict()

# Function to fetch the current stock price using yfinance for Indian stocks
def get_stock_price(ticker):
    try:
        # Assuming NSE symbols, append '.NS' to the ticker. Adjust if using BSE.
        stock = yf.Ticker(f"{ticker[4:]}.NS")  # Use '.BO' for BSE
        price = stock.history(period="1d")['Close'].iloc[-1]  # Get the latest closing price
        return round(float(price), 2)
    except Exception as e:
        print(f"Error fetching price for {ticker[4:]}: {e}")
        return None

net_quant = {key: round((buy_summary[key] - sell_summary[key]), 5) for key in sell_summary if round((buy_summary[key] - sell_summary[key]), 5) != 0}
difference_dict = {key: ((buy_price_dict[key]-sell_price_dict[key])/net_quant[key]) for key in net_quant}
# Calculate the remaining stock and get the current price
result = {}
for key in difference_dict:
    buy_quantity = buy_summary.get(key, 0)
    sell_quantity = sell_summary[key]
    remaining_quantity = buy_quantity - sell_quantity
    
    # Only process if remaining quantity is greater than zero
    if remaining_quantity != 0:
        buy_price = difference_dict.get(key, 0)
        current_price = get_stock_price(key)
        
        if current_price is not None:
            current_value = round(remaining_quantity * current_price, 2)
            unrealized_profit = current_value-(buy_price*net_quant[key])
            total_cost_of_remaining_shares = buy_price * remaining_quantity
            profit_percentage = (unrealized_profit / (buy_price*net_quant[key])) * 100 
            result[key] = [
                round(remaining_quantity, 10),  # Remaining quantity
                current_price,  # Current price of the stock
                round(buy_price,5), #price at which the stock was bought
                round(unrealized_profit, 2),  # Unrealized profit
                current_value,  # Current value of the portfolio/investment
                round(profit_percentage, 2)  # Profit percentage
            ]

# Print the result
print(result)


{'NSE:ABBOTINDIA': [0.68, 27053.95, 19403.41176, 5202.37, 18396.69, 39.43], 'NSE:BAJEL': [46.35, 266.15, 206.11823, 2782.47, 12336.05, 29.12], 'NSE:BBTC': [-0.005, 2272.3, 2425791.0, 12117.6, -11.36, -99.91], 'NSE:BIKAJI': [30.16, 835.35, 411.90821, 12771.01, 25194.16, 102.8], 'NSE:HCLTECH': [14.04, 1626.15, 1520.05698, 1489.55, 22831.15, 6.98], 'NSE:MAYURUNIQ': [24.32, 643.8, 508.91077, 3280.51, 15657.22, 26.51], 'NSE:METROPOLIS': [7.13, 1981.35, 988.71304, 7077.51, 14127.03, 100.4], 'NSE:MSUMI': [257.97, 69.01, 55.73838, 3423.68, 17802.51, 23.81], 'NSE:PRAJIND': [20.76, 700.7, 357.31792, 7128.61, 14546.53, 96.1], 'NSE:SRF': [6.22, 2491.75, 2735.22508, -1514.41, 15498.69, -8.9], 'NSE:WABAG': [8.31, 1299.55, 599.65945, 5816.09, 10799.26, 116.71], 'NSE:WIPRO': [29.22, 495.15, 344.42813, 4404.09, 14468.28, 43.76], 'NSE:YESBANK': [608.24, 23.99, 18.93675, 3073.59, 14591.68, 26.68]}


In [8]:
#this is the block of code that generates unrealized profits for companies for which portfolia holder has only bought stocks as
#well as bought and sold the stocks both.

import pandas as pd
from fpdf import FPDF
import yfinance as yf

# Load the Excel file
df_xlsx = pd.read_excel("Book2.xlsx")

# Filter buy and sell data
data_for_buy = df_xlsx[df_xlsx['TYPE'] == 'Buy']
data_for_sell = df_xlsx[df_xlsx['TYPE'] == 'Sell']

# Summarize the quantities by ticker
buy_summary = data_for_buy.groupby('EXCHANGE CODE: TICKER')['UNITS'].sum().to_dict()
sell_summary = data_for_sell.groupby('EXCHANGE CODE: TICKER')['UNITS'].sum().to_dict()
buy_price_dict = data_for_buy.groupby('EXCHANGE CODE: TICKER')['AMOUNT'].sum().to_dict()
sell_price_dict = data_for_sell.groupby('EXCHANGE CODE: TICKER')['AMOUNT'].sum().to_dict()


'''generates a table of all the stock holdings'''
# Function to fetch the current stock price using yfinance for Indian stocks
def get_stock_price(ticker):
    try:
        # Assuming NSE symbols, append '.NS' to the ticker. Adjust if using BSE.
        stock = yf.Ticker(f"{ticker[4:]}.NS") if ticker[0:3] == 'NSE' else yf.Ticker(f"{ticker[4:]}.BO") 
        price = stock.history(period="1d")['Close'].iloc[-1]  # Get the latest closing price
        return round(float(price), 2)
    except Exception as e:
        print(f"Error fetching price for {ticker[4:]}: {e}")
        return None

net_quant = {key: round((buy_summary[key] - sell_summary.get(key, 0)), 5) for key in buy_summary if round((buy_summary[key] - sell_summary.get(key, 0)), 5) != 0}
average_buy_price_dict = {key: ((buy_price_dict[key]-sell_price_dict.get(key, 0))/net_quant[key]) for key in net_quant }
total_unrealized_profit = 0
# Calculate the remaining stock and get the current price
result = {}
for key in average_buy_price_dict:
    buy_quantity = buy_summary.get(key, 0)
    sell_quantity = sell_summary.get(key, 0)
    remaining_quantity = buy_quantity - sell_quantity
    
    # Only process if remaining quantity is greater than zero
    if remaining_quantity != 0:
        buy_price = average_buy_price_dict.get(key, 0)
        current_price = get_stock_price(key)
        
        if current_price is not None:
            current_value = round(remaining_quantity * current_price, 2)
            unrealized_profit = current_value - (buy_price * remaining_quantity)
            total_cost_of_remaining_shares = buy_price * remaining_quantity
            profit_percentage = (unrealized_profit / total_cost_of_remaining_shares) * 100 
            total_unrealized_profit += unrealized_profit
            result[key] = [
                round(remaining_quantity, 10),  # Remaining quantity
                current_price,  # Current price of the stock
                round(buy_price,5),  # Price at which the stock was bought
                round(unrealized_profit, 2),  # Unrealized profit
                current_value,  # Current value of the portfolio/investment
                round(profit_percentage, 2)  # Profit percentage
            ]

# Print the result
print(result)

{'NSE:ABBOTINDIA': [0.68, 27053.95, 19403.41176, 5202.37, 18396.69, 39.43], 'NSE:APOLLOHOSP': [3.6, 6515.85, 5972.0, 1957.86, 23457.06, 9.11], 'NSE:ASHOKLEY': [66.85, 246.45, 224.23897, 1484.81, 16475.18, 9.91], 'NSE:BAJEL': [46.35, 266.15, 206.11823, 2782.47, 12336.05, 29.12], 'NSE:BBTC': [-0.005, 2272.3, 2425791.0, 12117.59, -11.36, -99.91], 'NSE:BIKAJI': [30.16, 835.35, 411.90821, 12771.01, 25194.16, 102.8], 'NSE:CENTURYPLY': [21.42, 730.6, 691.9, 828.95, 15649.45, 5.59], 'NSE:HAVELLS': [15.42, 1845.95, 1620.0, 3484.15, 28464.55, 13.95], 'NSE:HCLTECH': [14.04, 1626.15, 1520.05698, 1489.55, 22831.15, 6.98], 'NSE:IEX': [92.52, 185.81, 148.0, 3498.18, 17191.14, 25.55], 'NSE:KSCL': [15.4, 1047.0, 832.17403, 3308.32, 16123.8, 25.82], 'NSE:MARUTI': [1.78, 12205.65, 12793.14607, -1045.74, 21726.06, -4.59], 'NSE:MAYURUNIQ': [24.32, 643.8, 508.91077, 3280.51, 15657.22, 26.51], 'NSE:METROPOLIS': [7.13, 1981.35, 988.71304, 7077.51, 14127.03, 100.4], 'NSE:MSUMI': [257.97, 69.01, 55.73838, 3423.

In [4]:
'''generating the introductory data for the client ie the current market value and the total investment value and other things'''
total_investment_value_data = (df_xlsx[df_xlsx['TYPE']=='Cash Deposit'])
val = list((total_investment_value_data.groupby('EXCHANGE CODE: TICKER')['PRICE'].sum().to_dict()).values())
total_investment_value = val[0] #gets the total amount of money that is invested by the portfolio holder
print(total_investment_value)

value_buy_sum = data_for_buy['Value'].sum()
value_sell_sum = data_for_sell['Value'].sum()
total_value = round(value_buy_sum + value_sell_sum,4)

cash_balance_buy_sum = data_for_buy['Cash Balance'].sum()
cash_balance_sell_sum = data_for_sell['Cash Balance'].sum()
total_cash_balance = round(cash_balance_buy_sum + cash_balance_sell_sum,4)

market_value = round(total_cash_balance + total_value,4) # gets the total market value
print(market_value)

100000.0
12141747.3799


In [28]:
'''generating the booked and total profit/loss'''
net_quant_is_zero_dict = {key: round((buy_summary[key] - sell_summary.get(key, 0)), 5) for key in buy_summary if round((buy_summary[key] - sell_summary.get(key, 0)), 5) == 0}
# Calculate the remaining stock and get the current price
result_with_remaining_zero = {}
for key in net_quant_is_zero_dict:
    buy_quantity = buy_summary.get(key, 0)
    sell_quantity = sell_summary.get(key, 0)
    remaining_quantity = buy_quantity - sell_quantity
    buy_price = buy_price_dict.get(key, 0)
    current_price = get_stock_price(key)
    sell_price = sell_price_dict.get(key, 0)
    if current_price is not None:
        current_value = round(remaining_quantity * current_price, 2)
        booked_profit = (sell_price - buy_price) 
        result_with_remaining_zero[key] = round(booked_profit, 2)  # booked profit

# calculating the total booked profit 
total_booked_profit = sum(list(result_with_remaining_zero.values()))
# Print the result with remaining quantity zero
print(total_booked_profit)
print(total_unrealized_profit)
total_profit_or_loss = round(total_booked_profit  + total_unrealized_profit,2)
print(total_profit_or_loss)

211263.69
80153.06289999756
291416.75


In [30]:
from fpdf import FPDF
from babel.numbers import format_decimal

# Initialize the PDF
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=10)
pdf.add_page()

# Add a background color for the title
pdf.set_fill_color(200, 220, 255)
pdf.set_font("Arial", "B", 12)  # Bold font for the title
pdf.cell(0, 12, txt="Stock Portfolio Summary", ln=True, align="C", fill=True)
pdf.ln(10)
pdf.set_right_margin(10)

# Add introductory data with bold labels
pdf.set_font("Arial", "B", 10)
pdf.set_text_color(0, 0, 0)

# Format the numbers with Indian commas
formatted_investment_value = format_decimal(total_investment_value, locale='en_IN')
formatted_market_value = format_decimal(market_value, locale='en_IN')
formatted_booked_profit = format_decimal(total_booked_profit, locale='en_IN')
formatted_total_profit = format_decimal(total_profit_or_loss, locale='en_IN')

# Total Investment Value and Market Value
pdf.cell(50, 10, txt="Total Investment Value:", border=0, align="L")
pdf.set_font("Arial", "", 10)
pdf.cell(50, 10, txt=formatted_investment_value, border=0, align="L")
pdf.set_font("Arial", "B", 10)
pdf.cell(50, 10, txt="Market Value:", border=0, align="L")
pdf.set_font("Arial", "", 10)
pdf.cell(50, 10, txt=formatted_market_value, border=0, align="L")
pdf.ln(10)

# Booked Profit/Loss and Total Profit/Loss
pdf.set_font("Arial", "B", 10)
pdf.cell(50, 10, txt="Booked Profit/Loss:", border=0, align="L")
pdf.set_font("Arial", "", 10)
pdf.cell(50, 10, txt=formatted_booked_profit, border=0, align="L")
pdf.set_font("Arial", "B", 10)
pdf.cell(50, 10, txt="Total Profit/Loss:", border=0, align="L")
pdf.set_font("Arial", "", 10)
pdf.cell(50, 10, txt=formatted_total_profit, border=0, align="L")
pdf.ln(20)

# Set column headers with a background color and border, bold text
pdf.set_fill_color(105, 105, 105)  # Dark gray
pdf.set_text_color(255, 255, 255)  # White text for headers
pdf.set_font("Arial", "B", 10)  # Bold font for headers

# Adjust the width of each column so that it fits within the page
pdf.cell(25, 10, txt="Stock", border=1, align="C", fill=True)
pdf.cell(30, 10, txt="Net Quantity", border=1, align="C", fill=True)
pdf.cell(25, 10, txt="Buy Price", border=1, align="C", fill=True)
pdf.cell(25, 10, txt="Current Price", border=1, align="C", fill=True)
pdf.cell(25, 10, txt="Profit/Loss", border=1, align="C", fill=True)
pdf.cell(30, 10, txt="Profit/Loss%", border=1, align="C", fill=True)
pdf.cell(35, 10, txt="Current Value", border=1, align="C", fill=True)
pdf.ln()

# Add data rows
pdf.set_font("Arial", "", 10)  # Standard font size for data rows
pdf.set_text_color(0, 0, 0)  # Black text for data

for idx, (key, value) in enumerate(result.items()):
    # Alternate row background color for better readability
    if idx % 2 == 0:
        pdf.set_fill_color(245, 245, 245)  # Light gray
        fill = True
    else:
        fill = False

    # Add stock name and quantities
    pdf.cell(25, 10, txt=key[4:], border=1, align="C", fill=fill)  # Stock name without the exchange prefix
    pdf.cell(30, 10, txt=format_decimal(value[0], locale='en_IN'), border=1, align="C", fill=fill)  # Total quantities
    
    # Add buy price
    pdf.cell(25, 10, txt=format_decimal(value[2], locale='en_IN'), border=1, align="C", fill=fill)  # Buy price
    
    # Add current price
    pdf.cell(25, 10, txt=format_decimal(value[1], locale='en_IN'), border=1, align="C", fill=fill)  # Current price

    # Add profit with color coding
    if value[3] >= 0:
        pdf.set_text_color(0, 128, 0)  # Green for profit
    else:
        pdf.set_text_color(255, 0, 0)  # Red for loss
    pdf.cell(25, 10, txt=format_decimal(value[3], locale='en_IN'), border=1, align="C", fill=fill)  # Profit/Loss
    pdf.set_text_color(0, 0, 0)  # Reset text color to black
    
    # Add profit percentage with color coding
    if value[5] >= 0:
        pdf.set_text_color(0, 128, 0)  # Green for positive percentage
    else:
        pdf.set_text_color(255, 0, 0)  # Red for negative percentage
    pdf.cell(30, 10, txt=f"{value[5]:,.2f}%", border=1, align="C", fill=fill)  # Profit percentage
    
    # Reset text color to black and add current value
    pdf.set_text_color(0, 0, 0)
    pdf.cell(35, 10, txt=format_decimal(value[4], locale='en_IN'), border=1, align="C", fill=fill)  # Current value
    pdf.ln()

# Add a footer
pdf.set_y(-15)
pdf.set_font("Arial", "I", 8)  # Footer font size
pdf.set_text_color(128, 128, 128)
pdf.cell(0, 10, txt="End of Report", align="C")

# Save the PDF to a file
pdf.output("stock_portfolio2.pdf")

print("PDF created successfully.")


UnicodeEncodeError: 'latin-1' codec can't encode character '\u20b9' in position 275: ordinal not in range(256)